In [5]:
import os
import sys
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

current_dir = os.getcwd()
utils_path = os.path.join(current_dir, '..', 'utils')
utils_abs_path = os.path.abspath(utils_path)

if utils_abs_path not in sys.path:
    sys.path.append(utils_abs_path)

import get_data

DATA_PATH = get_data.get_dataset_abspath()

training_setA_path = DATA_PATH + 'training_setA'
training_setB_path = DATA_PATH + 'training_setB'

In [6]:
nan_counts = {}

# List all files in the directory
file_list = os.listdir(training_setA_path)

for file_name in file_list:
    if file_name.endswith(".psv"):
        file_path = os.path.join(training_setA_path, file_name)
        # Read the file
        data = pd.read_csv(file_path, sep='|')
        
        # For the first file, initialize the dictionary with column names
        if not nan_counts:
            for column in data.columns:
                nan_counts[column] = {"nan": 0, "total": 0}
        
        # Update NaN counts and total counts
        for column in data.columns:
            nan_counts[column]["nan"] += data[column].isna().sum()
            nan_counts[column]["total"] += len(data)

print(nan_counts)

{'HR': {'nan': 61189, 'total': 790215}, 'O2Sat': {'nan': 95079, 'total': 790215}, 'Temp': {'nan': 523314, 'total': 790215}, 'SBP': {'nan': 120201, 'total': 790215}, 'MAP': {'nan': 80858, 'total': 790215}, 'DBP': {'nan': 380297, 'total': 790215}, 'Resp': {'nan': 77258, 'total': 790215}, 'EtCO2': {'nan': 790215, 'total': 790215}, 'BaseExcess': {'nan': 707834, 'total': 790215}, 'HCO3': {'nan': 726598, 'total': 790215}, 'FiO2': {'nan': 678060, 'total': 790215}, 'pH': {'nan': 699600, 'total': 790215}, 'PaCO2': {'nan': 720927, 'total': 790215}, 'SaO2': {'nan': 751055, 'total': 790215}, 'AST': {'nan': 778395, 'total': 790215}, 'BUN': {'nan': 725739, 'total': 790215}, 'Alkalinephos': {'nan': 778683, 'total': 790215}, 'Calcium': {'nan': 750897, 'total': 790215}, 'Chloride': {'nan': 724438, 'total': 790215}, 'Creatinine': {'nan': 737728, 'total': 790215}, 'Bilirubin_direct': {'nan': 789033, 'total': 790215}, 'Glucose': {'nan': 693559, 'total': 790215}, 'Lactate': {'nan': 763072, 'total': 790215}